# PSO – Particle Swarm Optimization (részletes, magyarázatos verzió)

Ebben a notebookban lépésről lépésre van felépítve a **PSO (Particle Swarm Optimization)** algoritmus.
Minden feladat előtt találsz egy rövid **elméleti magyarázatot** és utána jöhetnek a kódok, ahol a `TODO` részeket neked kell kitöltened.

A PSO egy **populáció alapú, metaheurisztikus optimalizáló algoritmus**, amely folytonos keresési térben keres minimumot vagy maximumot.

---

## 1. feladat – Tesztfüggvények definiálása 2D-ben

Először definiálunk néhány klasszikus **benchmark függvényt**, amelyeken kipróbáljuk a PSO-t.

### 1.1 Sphere függvény

A legegyszerűbb tesztfüggvény a *Sphere*:

\[
f(x, y) = x^2 + y^2
\]

Tulajdonságok:

- konvex,
- egyetlen globális minimuma van: \( (x^*, y^*) = (0, 0) \),
- a minimum értéke: \( f(0,0) = 0 \).

### 1.2 Rastrigin függvény

A *Rastrigin* egy **nehéz** tesztfüggvény, sok lokális minimummal:

\[
f(x, y) = 20
+ x^2 - 10 \cos(2\pi x)
+ y^2 - 10 \cos(2\pi y)
\]

Tulajdonságok:

- nagyon sok lokális minimum → jó vizsga a PSO-nak,
- globális minimum: \( (0,0) \),
- globális minimum értéke: \( f(0,0) = 0 \).

### Feladat

Implementáld a két függvényt úgy, hogy a bemenet egy **2 elemű lista** legyen: `[x, y]`.

In [ ]:
import math

# TODO: Sphere függvény
def sphere(position):
    """Egyszerű konvex függvény: f(x,y) = x^2 + y^2."""
    # position = [x, y]
    # TODO: számítsd ki és add vissza az értékét
    raise NotImplementedError


# TODO: Rastrigin függvény
def rastrigin(position):
    """Sok lokális minimummal rendelkező tesztfüggvény."""
    # position = [x, y]
    # TODO: implementáld a képletet
    raise NotImplementedError


# Ellenőrzéshez (ha készen vagy):
# print("Sphere(0,0) =", sphere([0,0]))
# print("Rastrigin(0,0) =", rastrigin([0,0]))

---

## 2. feladat – A raj (swarm) inicializálása

A PSO-ban nem egyetlen megoldást tartunk nyilván, hanem **sok részecskét** (*particle*).  
Minden részecskére eltároljuk:

- aktuális pozíció: \( x_i^t \in \mathbb{R}^d \),
- aktuális sebesség: \( v_i^t \in \mathbb{R}^d \),
- saját legjobb pozíció: \( p_i \) (ahol eddig a legjobb értéket kapta).

A kezdeti pozíciókat tipikusan véletlenül választjuk:

\[
x_{i,j}^0 \sim U(\text{lower},\, \text{upper})
\]

A sebességek kezdő értékét is választhatjuk véletlenül, pl.:

\[
v_{i,j}^0 \sim U(-|upper-lower|,\, |upper-lower|)
\]

### Feladat

1. Írj függvényt, amely egy `dim` hosszú **véletlen vektort** generál a megadott intervallumban.
2. Írj `init_swarm()` függvényt, amely:
   - létrehozza a pozíciókat és sebességeket,
   - kiszámítja a kezdeti **pbest pozíciókat és értékeket** egy megadott célfüggvényre.

In [ ]:
import random

# TODO: véletlen vektor generálása
def random_vector(dim, lower, upper):
    """Dimenzió: dim, minden komponens ~ U(lower, upper)."""
    # TODO: térj vissza egy listával (pl. [x1, x2, ..., x_dim])
    raise NotImplementedError


# TODO: raj inicializálása
def init_swarm(num_particles, dim, lower, upper, objective_fn):
    """
    Visszatérés:
      positions        – lista (num_particles darab dim-dimenziós pozíció)
      velocities       – lista (num_particles darab dim-dimenziós sebesség)
      pbest_positions  – kezdetben = positions
      pbest_values     – objective_fn(positions[i]) értékek
    """
    # TODO: hozd létre ezeket a struktúrákat és add vissza őket
    raise NotImplementedError


# Példa (ha majd készen vagy):
# positions, velocities, pbest_pos, pbest_val = init_swarm(10, 2, -5, 5, sphere)
# print(len(positions), len(velocities))

---

## 3. feladat – Swarm kiértékelése, személyes és globális legjobb pozíciók

Minden iterációban ki kell értékelni az összes részecske aktuális pozícióját:

\[
f_i^t = f(x_i^t)
\]

A személyes legjobb (pbest) frissítése:

\[
\text{ha } f(x_i^t) < f(p_i), \quad \text{akkor } p_i := x_i^t
\]

A globális legjobb (gbest) frissítése:

\[
g := \arg\min_i f(p_i)
\]

### Feladat

Implementáld az `evaluate_swarm()` függvényt, amely:

- bejárja az aktuális `positions` listát,
- frissíti a `pbest_positions` és `pbest_values` listákat,
- meghatározza és visszaadja a `gbest_position` és `gbest_value` értékeit.

In [ ]:
# TODO: Swarm kiértékelése és pbest / gbest frissítése
def evaluate_swarm(positions, pbest_positions, pbest_values, objective_fn):
    """
    positions: aktuális pozíciók
    pbest_positions, pbest_values: eddigi legjobb értékek és pozíciók
    objective_fn: minimalizálandó függvény

    Visszatérés:
      pbest_positions, pbest_values, gbest_position, gbest_value
    """
    # TODO: járd végig a pozíciókat, frissítsd a pbest-et, keresd meg a globális minimumot
    raise NotImplementedError

---

## 4. feladat – Sebesség- és pozíciófrissítés (PSO dinamika)

A PSO kulcslépése a **sebesség- és pozíciófrissítés**.

### 4.1 Sebességfrissítés

Az i-edik részecske sebessége:

\[
v_i^{t+1} = w v_i^t
+ c_1 r_1 (p_i - x_i^t)
+ c_2 r_2 (g - x_i^t)
\]

- \( w \) – tehetetlenségi súly (inertia weight)
- \( c_1 \) – kognitív (saját) komponens súlya
- \( c_2 \) – szociális (raj) komponens súlya
- \( r_1, r_2 \sim U(0,1) \) véletlen skálázó tényezők

Opcionálisan bevezethetünk **sebességkorlátot**:

\[
v_{i,j}^{t+1} := \max(-v_{\max}, \min(v_{\max}, v_{i,j}^{t+1}))
\]

### 4.2 Pozíciófrissítés

Az i-edik részecske pozíciója:

\[
x_i^{t+1} = x_i^t + v_i^{t+1}
\]

Majd a pozíciót visszavágjuk a megadott tartományba:

\[
x_{i,j}^{t+1} := \max(lower, \min(upper, x_{i,j}^{t+1}))
\]

### Feladat

Implementáld a két függvényt:

- `update_velocities()` – az előző képletek alapján,
- `update_positions()` – egyszerű vektorösszeadás + határok közé szorítás.

In [ ]:
# TODO: sebességfrissítés
def update_velocities(positions, velocities, pbest_positions, gbest_position,
                      w=0.7, c1=1.5, c2=1.5, vmax=None):
    """Frissítsd az összes részecske sebességét a PSO-képlettel."""
    # Tipp:
    # - járd végig a részecskéket (for i in range(len(positions)))
    # - generálj r1, r2 véletlen számokat [0,1] intervallumban
    # - számítsd ki a new_v komponenseit dimenziónként
    # - ha vmax nem None, vágd le a sebességet
    raise NotImplementedError


# TODO: pozíciófrissítés
def update_positions(positions, velocities, lower, upper):
    """Frissítsd a pozíciókat a sebességek alapján, és vágd vissza a tartományba."""
    # Tipp:
    # - new_x = x + v komponensenként
    # - lower <= new_x[j] <= upper
    raise NotImplementedError

---

## 5. feladat – Teljes PSO algoritmus összeállítása

Most már rendelkezésre áll minden építőkocka:

1. `init_swarm()`
2. `evaluate_swarm()`
3. `update_velocities()`
4. `update_positions()`

### A teljes algoritmus menete

**Inicializálás:**
- Swarm létrehozása (pozíciók, sebességek, pbest-ek, gbest).

**Minden iterációban:**

1. Kiértékelés: `evaluate_swarm()`  
2. Sebességfrissítés: `update_velocities()`  
3. Pozíciófrissítés: `update_positions()`  
4. Eltároljuk a legjobb értéket egy `history` listában, hogy később vissza tudjuk rajzolni a konvergenciát.

### Feladat

Implementáld a `run_pso()` függvényt, amely:

- paraméterként megkapja a célfüggvényt (pl. `sphere` vagy `rastrigin`),
- visszaadja:
  - a legjobb megtalált pozíciót (`gbest_position`),
  - a legjobb értéket (`gbest_value`),
  - a konvergenciagörbét (`history` – legjobb érték iterációnként).

In [ ]:
# TODO: teljes PSO futtatása
def run_pso(objective_fn, num_particles=30, dim=2, lower=-5, upper=5,
            w=0.7, c1=1.5, c2=1.5, vmax=None, n_iters=100):
    """
    objective_fn: minimalizálandó függvény (pl. sphere vagy rastrigin)
    Vissza:
      gbest_position, gbest_value, history
    """
    # Lépések (javaslat):
    # 1. init_swarm(...)
    # 2. history = [kezdeti gbest_value]
    # 3. for iter in range(n_iters):
    #       - evaluate_swarm(...)
    #       - update_velocities(...)
    #       - update_positions(...)
    #       - mentsd el az aktuális gbest_value-t a history-ba
    raise NotImplementedError

---

## 6. feladat – Konvergenciagörbe rajzolása

A PSO viselkedését jól lehet szemléltetni az ún. **konvergenciagörbével**:

- a vízszintes tengelyen az iterációk száma,
- a függőleges tengelyen a legjobb érték \( f(g^t) \).

Ha az algoritmus működik, akkor a görbe **csökkenő trendet** mutat.

### Feladat

Implementáld a `plot_convergence()` függvényt.

In [ ]:
import matplotlib.pyplot as plt

def plot_convergence(history, title="PSO konvergencia"):
    """Rajzold ki a history listát (legjobb érték iterációnként)."""
    # TODO: használj plt.plot(...), adj meg tengelyfeliratokat és címet
    raise NotImplementedError